In [1]:
# Ran on GCP Deep Learning VM
!git clone https://github.com/ChengBinJin/SpineC2M.git

Cloning into 'SpineC2M'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 453 (delta 0), reused 0 (delta 0), pack-reused 450
Receiving objects: 100% (453/453), 103.31 KiB | 0 bytes/s, done.
Resolving deltas: 100% (294/294), done.


In [36]:
#%cd SpineC2M/DC2Anet/

## Run this in SpineC2M/DC2Anet/

from pathlib import Path
from zipfile import ZipFile
import random
from PIL import Image

# Creating custom dataset with RIREImageDataset
# Change path to the path where you have the RIREImageDataset_Clean.zip
dataset_path = "../../RIREImageDataset_Clean_2.zip"

# Change path to where you want your images to be located (note: if you change you'll probably have to change subsequent code as well)
output_path = "./datasets/RIRE"
ct_folder = "RIRECT"
mr_folder = "RIREMRI_T1" # Change to RIREMRI_T2 or RIREMRI_PD if necessary

# Create the image folders
# A: CT Scans
# B: MR Scans
Path("{}".format(output_path)).mkdir(parents=True, exist_ok=True)
Path("{}/trainA".format(output_path)).mkdir(parents=True, exist_ok=True)
Path("{}/testA".format(output_path)).mkdir(parents=True, exist_ok=True)
Path("{}/trainB".format(output_path)).mkdir(parents=True, exist_ok=True)
Path("{}/testB".format(output_path)).mkdir(parents=True, exist_ok=True)

# Unzip specific folders
# Do a test-train split of 70:30
train_split = 0.7

# Create a ZipFile Object and load sample.zip in it
with ZipFile(dataset_path, 'r') as zipObj:
   # Get a list of all archived file names from the zip
   listOfFileNames = zipObj.namelist()
   # Iterate over the file names
   # print(listOfFileNames)
   ct_files = [ctImg for ctImg in listOfFileNames if ct_folder in ctImg and ".jpg" in ctImg]
   mr_files = [mrImg for mrImg in listOfFileNames if mr_folder in mrImg and ".jpg" in mrImg]

   print(ct_files)
   print(mr_files)

   # Shuffle and randomize the images
   random.shuffle(ct_files)
   random.shuffle(mr_files)

   # Get the training/testing index for split
   splitIndexA = int(train_split * len(ct_files))
   splitIndexB = int(train_split * len(mr_files))

   # Train A
   for imagePath in ct_files[:splitIndexA]:
     with zipObj.open(imagePath) as file:
       im = Image.open(file)
       im.save('{}/trainA/{}'.format(output_path, imagePath.split('/')[1]))
  
   # Test A
   for imagePath in ct_files[splitIndexA:]:
      with zipObj.open(imagePath) as file:
       im = Image.open(file)
       im.save('{}/testA/{}'.format(output_path, imagePath.split('/')[1]))

   # Train B
   for imagePath in mr_files[:splitIndexB]:
      with zipObj.open(imagePath) as file:
       im = Image.open(file)
       im.save('{}/trainB/{}'.format(output_path, imagePath.split('/')[1]))

   # Test B
   for imagePath in mr_files[splitIndexB:]:
      with zipObj.open(imagePath) as file:
       im = Image.open(file)
       im.save('{}/testB/{}'.format(output_path, imagePath.split('/')[1]))

/home/jupyter/SpineC2M/DC2Anet
['RIRECT/patient_001_ct_0.jpg', 'RIRECT/patient_001_ct_1.jpg', 'RIRECT/patient_001_ct_10.jpg', 'RIRECT/patient_001_ct_11.jpg', 'RIRECT/patient_001_ct_12.jpg', 'RIRECT/patient_001_ct_13.jpg', 'RIRECT/patient_001_ct_14.jpg', 'RIRECT/patient_001_ct_15.jpg', 'RIRECT/patient_001_ct_16.jpg', 'RIRECT/patient_001_ct_17.jpg', 'RIRECT/patient_001_ct_18.jpg', 'RIRECT/patient_001_ct_19.jpg', 'RIRECT/patient_001_ct_2.jpg', 'RIRECT/patient_001_ct_20.jpg', 'RIRECT/patient_001_ct_21.jpg', 'RIRECT/patient_001_ct_22.jpg', 'RIRECT/patient_001_ct_23.jpg', 'RIRECT/patient_001_ct_24.jpg', 'RIRECT/patient_001_ct_25.jpg', 'RIRECT/patient_001_ct_26.jpg', 'RIRECT/patient_001_ct_27.jpg', 'RIRECT/patient_001_ct_3.jpg', 'RIRECT/patient_001_ct_4.jpg', 'RIRECT/patient_001_ct_5.jpg', 'RIRECT/patient_001_ct_6.jpg', 'RIRECT/patient_001_ct_7.jpg', 'RIRECT/patient_001_ct_8.jpg', 'RIRECT/patient_001_ct_9.jpg', 'RIRECT/patient_002_ct_0.jpg', 'RIRECT/patient_002_ct_1.jpg', 'RIRECT/patient_002_

In [37]:
## Run this in SpineC2M/DC2Anet/
#%cd DC2Anet

# To convert to Paired TFRecords
# Spine2CM Requires the MR/CT images be combined side by side
# Based on https://stackoverflow.com/questions/10647311/how-do-you-merge-images-into-a-canvas-using-pil-pillow
# Need to make sure that the image is resized- default 256, 256
from PIL import Image

# For TFRecords
Path("{}/TFTrain".format(output_path)).mkdir(parents=True, exist_ok=True)
Path("{}/TFTest".format(output_path)).mkdir(parents=True, exist_ok=True)

def combineCT_MR(ct_files, mr_files, savePath, image_dim=(256, 256)):
  imgPairs = []
  for imgPath in ct_files:
    # print(imgPath)
    # Get Patient ID
    CTJPG = imgPath.split('/')[-1].split(".jpg")[0]
    # print(CTJPG)
    imgInfo = CTJPG.split("_ct_") # Patient ID at index 0, Slice number at index 1
    # print(imgInfo)

    MRJPG = [mrPath for mrPath in mr_files if "{}_mr_T1_{}.jpg".format(imgInfo[0], imgInfo[1]) in mrPath]

    # Note: If there is no original image, we use rectified image
    if len(MRJPG) == 0:
      MRJPG = [mrPath for mrPath in mr_files if "{}_mr_T1_rectified_{}.jpg".format(imgInfo[0], imgInfo[1]) in mrPath]

    # print(MRJPG)

    # If we have a match, then we read both images and then we combine them (resize then merge)
    # We expect each jpg to be numbered uniquely, so it should only contain 1 element
    if MRJPG and len(MRJPG) == 1:
      # Resize both ct and mr scan
      with ZipFile(dataset_path, 'r') as zipObj:
        with zipObj.open(imgPath) as ctFile:
          ct_img = Image.open(ctFile) #cv2.imread(imgPath, 0)
          
          with zipObj.open(MRJPG[0]) as mrFile:
            mr_img = Image.open(mrFile) #cv2.imread(MRJPG[0], 0)

            # print(ct_img.size)

            merge_img = Image.new('RGB', (image_dim[0]*2,image_dim[1]))

            ct_img = ct_img.resize(image_dim)
            mr_img = mr_img.resize(image_dim)

            # First paste the mr_image on the left, and then the ct_image on the right
            merge_img.paste(mr_img, (0,0))
            merge_img.paste(ct_img, (image_dim[0], 0))

            merge_img.show()

            pathName = "{}/{}.jpg".format(savePath, "_merged_".join(imgInfo))
            merge_img.save(pathName)

            # print(pathName)
            imgPairs.append(pathName)

  return imgPairs

trainSplit = 0.7
trainSplitIndex = int(trainSplit * len(ct_files))

random.shuffle(ct_files)

imgTrain = combineCT_MR(ct_files[:trainSplitIndex], mr_files, "{}/TFTrain".format(output_path))
print("{} images merged from {} and {}".format(len(imgTrain), ct_folder, mr_folder))
imgTest = combineCT_MR(ct_files[trainSplitIndex:], mr_files, "{}/TFTest".format(output_path))
print("{} images merged from {} and {}".format(len(imgTest), ct_folder, mr_folder))

130 images merged from RIRECT and RIREMRI_T1
70 images merged from RIRECT and RIREMRI_T1


In [38]:
# %cd ../
# After you build the tfrecords, you can move the files to any desired location (provided it matches the filepath you use for training/testing)
!python build_data.py --input_dataA DC2Anet/datasets/RIRE/TFTrain --input_dataB DC2Anet/datasets/RIRE/TFTest

/home/jupyter/SpineC2M

Convert DC2Anet/datasets/RIRE/TFTrain data to tfrecords...
input_dir: DC2Anet/datasets/RIRE/TFTrain

W0530 21:13:44.621717 140642290926976 module_wrapper.py:139] From build_data.py:32: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

Instructions for updating:
Use tf.gfile.GFile.
W0530 21:13:44.622651 140642290926976 deprecation.py:323] From build_data.py:37: FastGFile.__init__ (from tensorflow.python.platform.gfile) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.gfile.GFile.
Processed 0/130...
Processed 100/130...
Finished!
Convert DC2Anet/datasets/RIRE/TFTest data to tfrecords...
input_dir: DC2Anet/datasets/RIRE/TFTest
Processed 0/70...
Finished!


In [ ]:
# Create a new dataset.py file that can be used for the specific network you want to train
# Modified based on original code from https://github.com/ChengBinJin/SpineC2M to fit our use case
%%writefile dataset.py

# ---------------------------------------------------------
# Tensorflow SpineC2M-discogan Implementation
# Licensed under The MIT License [see LICENSE for details]
# Written by Cheng-Bin Jin
# Email: sbkim0407@gmail.com
# ---------------------------------------------------------
import os
import sys
import logging

logger = logging.getLogger(__name__)  # logger
logger.setLevel(logging.INFO)


def _init_logger(flags, log_path):
    if flags.is_train:
        formatter = logging.Formatter('%(asctime)s:%(name)s:%(message)s')
        # file handler
        file_handler = logging.FileHandler(os.path.join(log_path, 'dataset.log'))
        file_handler.setFormatter(formatter)
        file_handler.setLevel(logging.INFO)
        # stream handler
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)
        # add handlers
        logger.addHandler(file_handler)
        logger.addHandler(stream_handler)


class SpineC2M(object):
    def __init__(self, flags):
        self.flags = flags
        self.name = 'day2night'
        self.image_size = (300, 200, 1)
        self.num_tests = 4426

        # tfrecord path
        self.train_tfpath = '../../Data/spine06/tfrecords/train.tfrecords'
        self.test_tfpath = '../../Data/spine06/tfrecords/test.tfrecords'

        logger.info('Initialize {} dataset SUCCESS!'.format(self.flags.dataset))
        logger.info('Img size: {}'.format(self.image_size))

    def __call__(self, is_train='True'):
        if is_train:
            if not os.path.isfile(self.train_tfpath):
                sys.exit(' [!] Train tfrecord file is not found...')
            return self.train_tfpath
        else:
            if not os.path.isfile(self.test_tfpath):
                sys.exit(' [!] Test tfrecord file is not found...')
            return self.test_tfpath

# CS168: Additional class objects to run the networks using our customized model
# To use this, you have to run this cell and then replace the dataset.py file in SpineC2M/[NETWORK NAME]/dataset.py with this file
# Modify the train_tfpath and test_tfpath as necessary
class Brain_T1(object):
    def __init__(self, flags):
        self.flags = flags
        self.name = 'brain_t1'
        self.image_size = (256, 256, 1)
        self.num_tests = 70

        # tfrecord path
        self.train_tfpath = '../../tfrecords/train.tfrecords'
        self.test_tfpath = '../../tfrecords/test.tfrecords'

        logger.info('Initialize {} dataset SUCCESS!'.format(self.flags.dataset))
        logger.info('Img size: {}'.format(self.image_size))

    def __call__(self, is_train='True'):
        if is_train:
            if not os.path.isfile(self.train_tfpath):
                sys.exit(' [!] Train tfrecord file is not found...')
            return self.train_tfpath
        else:
            if not os.path.isfile(self.test_tfpath):
                sys.exit(' [!] Test tfrecord file is not found...')
            return self.test_tfpath

class Brain_T2(object):
    def __init__(self, flags):
        self.flags = flags
        self.name = 'brain_t2'
        self.image_size = (256, 256, 1)
        self.num_tests = 70

        # tfrecord path
        self.train_tfpath = '../../tfrecords/train.tfrecords'
        self.test_tfpath = '../../tfrecords/test.tfrecords'

        logger.info('Initialize {} dataset SUCCESS!'.format(self.flags.dataset))
        logger.info('Img size: {}'.format(self.image_size))

    def __call__(self, is_train='True'):
        if is_train:
            if not os.path.isfile(self.train_tfpath):
                sys.exit(' [!] Train tfrecord file is not found...')
            return self.train_tfpath
        else:
            if not os.path.isfile(self.test_tfpath):
                sys.exit(' [!] Test tfrecord file is not found...')
            return self.test_tfpath

class Brain_PD(object):
    def __init__(self, flags):
        self.flags = flags
        self.name = 'brain_pd'
        self.image_size = (256, 256, 1)
        self.num_tests = 70

        # tfrecord path
        self.train_tfpath = '../../tfrecords/train.tfrecords'
        self.test_tfpath = '../../tfrecords/test.tfrecords'

        logger.info('Initialize {} dataset SUCCESS!'.format(self.flags.dataset))
        logger.info('Img size: {}'.format(self.image_size))

    def __call__(self, is_train='True'):
        if is_train:
            if not os.path.isfile(self.train_tfpath):
                sys.exit(' [!] Train tfrecord file is not found...')
            return self.train_tfpath
        else:
            if not os.path.isfile(self.test_tfpath):
                sys.exit(' [!] Test tfrecord file is not found...')
            return self.test_tfpath

# noinspection PyPep8Naming
def Dataset(dataset_name, flags, log_path=None):
    if flags.is_train:
        _init_logger(flags, log_path)  # init logger

    if dataset_name == 'spine06':
        return SpineC2M(flags)
    elif dataset_name == 'brain_t1':
        return Brain_T1(flags)
    elif dataset_name == 'brain_t2':
        return Brain_T2(flags)
    elif dataset_name == 'brain_pd':
        return Brain_PD(flags)
    else:
        raise NotImplementedError

In [ ]:
# Run this in SpineC2M/DC2ANet/ (or whichever GAN network you want to run)
# %cd SpineC2M/DC2Anet/

# BEFORE YOU RUN:
# 0. Download the caffe_layers_value.pickle from https://github.com/ChengBinJin/SpineC2M/issues/2
# 1. Change the dataset.py file in your desired GAN Network with the custom one created in the previous cell. Make sure self.train_tfpath and self.test_tfpath is correct!
# 2. In SpineC2M/[NETWORK NAME]/[NETWORK NAME].py (e.g. SpineC2M/DC2Anet/dc2anet.py), change line 13 to mpl.use('WebAgg')
# 3. In vgg16.py, change line 12 to weight_file_path = 'your/path/to/caffe_layers_value.pickle'
# 4. Add flags to !python main.py: --dataset [brain_t1 or brain_t2 or brain_pd] --iters [reasonable number] --print_freq 1 --save_freq 100 --sample_freq 100
#    (see main.py for more information)
# 5. In main.py, add this after you import os
#    os.environ['KMP_DUPLICATE_LIB_OK']='True'
# Use --load_model [MODEL FOLDER NAME] to resume training from a previous model

!python main.py --dataset brain_t1 --iters 15000 print_freq 1 --save_freq 100 --sample_freq 100 --load_model 20200530-2114

[Errno 2] No such file or directory: '../../DC2Anet/'
/home/jupyter/SpineC2M/DC2Anet

User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=4

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOC

In [17]:
# Evaluating the results for the model
# Execute this in your network folder (same as training script folder)
# %cd SpineC2M/DC2Anet

# BEFORE YOU RUN:
# 1. Set TF FLAG is_train to False in main.py (line 29)
# 2. In SpineC2M/[NETWORK NAME]/[NETWORK NAME].py (e.g. DC2Anet/dc2anet.py), configure the function plot_tests so that writes to correct folder
# E.G.
#    def plots_test(self, imgs, img_name, save_file, eval_file, gt_file):
#        num_imgs = len(imgs)
#
#        canvas = np.zeros((self.img_size[0], num_imgs * self.img_size[1]), np.uint8)
#        for idx in range(num_imgs):
#            canvas[:, idx * self.img_size[1]: (idx+1) * self.img_size[1]] = \
#                np.squeeze(255. * utils.inverse_transform(imgs[idx]))
#
#        img_name_ = img_name.astype('U26')[0]
#        # save imgs on test folder
#        # cv2.imwrite(os.path.join(save_file, img_name_), canvas)
#        # save imgs on eval folder
#        cv2.imwrite(os.path.join(eval_file, img_name_), canvas[:,self.img_size[1]:2*self.img_size[1]])
#        # save imgs on gt folder
#        cv2.imwrite(os.path.join(gt_file, img_name_), canvas[:, 2*self.img_size[1]:3*self.img_size[1]])

# 3. Use --load_model [MODEL FOLDER NAME] to evaluate the testing set using an existing pretrained model
!python main.py --dataset brain_t1 --load_model 20200530-2114


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=4

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BA

In [24]:
# Generates box plots and xlsx statistics file for all the networks you want to evaluate

# BEFORE YOU RUN:
# 1. In SpineC2M/eval/src/eval.py, change methods_ and display_names_ (lines 87, 88) based on the models that you want to test. 
# E.G. methods_ = ['DC2Anet', 'CycleGAN', 'DiscoGAN'] display_names_ = ['DC2Anet', 'CycleGAN', 'DiscoGAN']
# 2. In the same file, comment out lines 36 and 37 (which is basically a name check specifically for their dataset, but doesn't apply to RIRE image dataset)

# %cd SpineC2M/eval/src
!python eval.py

/home/jupyter/SpineC2M/eval/src

User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=4

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_

In [35]:
!zip -r CS168Results.zip Final

  adding: Final/ (stored 0%)
  adding: Final/MAE.jpg (deflated 44%)
  adding: Final/RMSE.jpg (deflated 43%)
  adding: Final/SSIM.jpg (deflated 43%)
  adding: Final/PCC.jpg (deflated 44%)
  adding: Final/statistics.xlsx (deflated 4%)
  adding: Final/PSNR.jpg (deflated 45%)
